In [1]:
import pandas as pd
import numpy as np
import cv2, random, os, shutil
from tqdm import tqdm
from PIL import Image

In [2]:
custom_data_frames_dir = "daycare_fall/Secaucus_data/merge_frames"
ava_data_frames_dir = "ava/frames"
custom_data_ann_dir = "daycare_fall/Secaucus_data/annotations"
ava_data_ann_dir = "ava/annotations"

combined_data_frames_dir = "combined/frames"
combined_data_ann_dir = "combined/annotations"

os.makedirs(combined_data_frames_dir, exist_ok=True)
os.makedirs(combined_data_ann_dir, exist_ok=True)

In [3]:
%%time
# Frame directories
frame_sources = [custom_data_frames_dir, ava_data_frames_dir]

print("📁 Starting to combine frame folders...\n")

for source_dir in frame_sources:
    print(f"🔍 Scanning: {source_dir}")
    items = os.listdir(source_dir)
    
    for item in tqdm(items, desc=f"📦 Copying from {os.path.basename(source_dir)}", unit="item"):
        src_path = os.path.join(source_dir, item)
        dst_path = os.path.join(combined_data_frames_dir, item)

        try:
            if os.path.isdir(src_path):
                shutil.copytree(src_path, dst_path, dirs_exist_ok=True)
                # print(f"📂 Folder copied: {item}")
            else:
                shutil.copy2(src_path, dst_path)
                # print(f"🖼️ File copied: {item}")
        except Exception as e:
            print(f"❌ Error copying {item}: {e}")

print("\n✅ Frame copying complete.")

📁 Starting to combine frame folders...

🔍 Scanning: daycare_fall/Secaucus_data/merge_frames


📦 Copying from merge_frames: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [01:10<00:00,  2.09s/item]


🔍 Scanning: ava/frames


📦 Copying from frames: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [04:24<00:00, 13.22s/item]


✅ Frame copying complete.
CPU times: user 26.8 s, sys: 1min 18s, total: 1min 45s
Wall time: 5min 35s


In [5]:
!ls "{custom_data_ann_dir}"

action_labels.pbtxt  excluded.csv      train_labels.csv
eval_frames.csv      train_frames.csv  val_labels.csv


In [6]:
!ls "{ava_data_ann_dir}"

ava_action_list_v2.2_for_activitynet_2019.pbtxt  excluded.csv
ava_train_tiny_v2.2.csv				 train_tiny.csv
ava_val_tiny_v2.2.csv				 val_tiny.csv


In [3]:
def parse_pbtxt(file_path):
    items = []
    with open(file_path, 'r') as f:
        item = {}
        for line in f:
            line = line.strip()
            if line.startswith("item"):
                item = {}
            elif line.startswith("name:"):
                item['name'] = line.split(":")[1].strip().strip('"')
            elif line.startswith("id:"):
                item['id'] = int(line.split(":")[1].strip())
                items.append(item)
    return items

In [4]:
dataset_to_ava_map = {
    "fall": [5],
    "standing": [12],
    "sitting": [11],
    "walking": [14],
    "crawling": [3],  # No exact match, but crouch/kneel is similar
    "sleeping": [8],
    "eating": [29],
    "jumping": [7],
    "others": [  # all remaining IDs
        1, 4, 6, 9, 10, 13, 15, 17, 20, 22, 24, 26, 27, 28,
        30, 34, 36, 37, 38, 41, 43, 45, 46, 47, 48, 49, 51,
        52, 54, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66,
        67, 68, 69, 70, 72, 73, 74, 76, 77, 78, 79, 80
    ]
}
ava_to_dataset_id_map = {
    5: 1,   # "fall down" → fall
    12: 2,  # "stand" → standing
    11: 3,  # "sit" → sitting
    14: 4,  # "walk" → walking
    3: 5,   # "crouch/kneel" → crawling (approximate match)
    8: 6,   # "lie/sleep" → sleeping
    29: 7,  # "eat" → eating
    7: 8    # "jump/leap" → jumping
}

In [5]:
ava_train_frames = pd.read_csv(os.path.join(ava_data_ann_dir, "train_tiny.csv"), sep=r'\s+', engine='python')
ava_val_frames = pd.read_csv(os.path.join(ava_data_ann_dir, "val_tiny.csv"), sep=r'\s+', engine='python')
custom_train_frames = pd.read_csv(os.path.join(custom_data_ann_dir, "train_frames.csv"), sep=r'\s+', engine='python')
custom_val_frames = pd.read_csv(os.path.join(custom_data_ann_dir, "eval_frames.csv"), sep=r'\s+', engine='python')

combined_train_frames = pd.concat([custom_train_frames, ava_train_frames])
combined_train_frames["labels"] = '""'
combined_val_frames = pd.concat([custom_val_frames]) #pd.concat([custom_val_frames, ava_val_frames])
combined_val_frames["labels"] = '""'

ava_classes = parse_pbtxt(os.path.join(ava_data_ann_dir, "ava_action_list_v2.2_for_activitynet_2019.pbtxt"))
ava_train_tiny_labels = pd.read_csv(os.path.join(ava_data_ann_dir, "ava_train_tiny_v2.2.csv"))
ava_val_tiny_labels = pd.read_csv(os.path.join(ava_data_ann_dir, "ava_train_tiny_v2.2.csv"))

custom_classes = parse_pbtxt(os.path.join(custom_data_ann_dir, "action_labels.pbtxt"))
custom_train_tiny_labels = pd.read_csv(os.path.join(custom_data_ann_dir, "train_labels.csv"))
custom_val_tiny_labels = pd.read_csv(os.path.join(custom_data_ann_dir, "val_labels.csv"))

ava_train_tiny_labels.columns = ["video_id", "timestamp", 
                                "xmin", "ymin", "xmax", "ymax",
                                "classes", "not_use"]
ava_train_tiny_labels = ava_train_tiny_labels.drop(["not_use"], axis=1)

custom_train_tiny_labels.columns = ["video_id", "timestamp", 
                                "xmin", "ymin", "xmax", "ymax",
                                "classes"]

ava_train_tiny_labels["classes"] = ava_train_tiny_labels["classes"].map(ava_to_dataset_id_map)

ava_train_tiny_labels = ava_train_tiny_labels.dropna()

ava_val_tiny_labels.columns = ["video_id", "timestamp", 
                                "xmin", "ymin", "xmax", "ymax",
                                "classes", "not_use"]
ava_val_tiny_labels = ava_val_tiny_labels.drop(["not_use"], axis=1)

custom_val_tiny_labels.columns = ["video_id", "timestamp", 
                                "xmin", "ymin", "xmax", "ymax",
                                "classes", "not_use"]

ava_val_tiny_labels["classes"] = ava_val_tiny_labels["classes"].map(ava_to_dataset_id_map)

ava_val_tiny_labels = ava_val_tiny_labels.dropna()

combined_train_labels = pd.concat([custom_train_tiny_labels, ava_train_tiny_labels]).reset_index(drop=True)
combined_val_labels = pd.concat([custom_val_tiny_labels]).reset_index(drop=True) #pd.concat([custom_val_tiny_labels, ava_val_tiny_labels]).reset_index(drop=True)

combined_train_labels["classes"] = combined_train_labels["classes"].astype(int)
combined_val_labels["classes"] = combined_val_labels["classes"].astype(int)
combined_val_labels["not_use"] = 1.0

In [7]:
combined_train_frames.to_csv(os.path.join(combined_data_ann_dir, "combined_train_frames.csv"), sep=" ", index=False, quoting=3)
combined_val_frames.to_csv(os.path.join(combined_data_ann_dir, "combined_val_frames.csv"), sep=" ", index=False, quoting=3)
combined_train_labels.to_csv(os.path.join(combined_data_ann_dir, "combined_train_labels.csv"), index=False, header=False)
combined_val_labels.to_csv(os.path.join(combined_data_ann_dir, "combined_val_labels.csv"), index=False, header=False)

In [8]:
shutil.copy(os.path.join(custom_data_ann_dir, "action_labels.pbtxt"), combined_data_ann_dir)
shutil.copy(os.path.join(ava_data_ann_dir, "excluded.csv"), combined_data_ann_dir)

'combined/annotations/excluded.csv'

In [17]:
custom_val_tiny_labels = pd.read_csv(os.path.join(custom_data_ann_dir, "val_labels.csv"))

custom_val_tiny_labels.columns = ["video_id", "timestamp", 
                                "xmin", "ymin", "xmax", "ymax",
                                "classes", "not_use"]

custom_to_ava_id_map = {y:x for x, y in ava_to_dataset_id_map.items()}
custom_val_tiny_labels["classes"] = custom_val_tiny_labels["classes"].map(custom_to_ava_id_map)
custom_val_tiny_labels.to_csv(os.path.join(custom_data_ann_dir, "val_labels_ava_mapped.csv"), index=False, header=False)

In [14]:
custom_val_tiny_labels

,video_id,timestamp,xmin,ymin,xmax,ymax,classes,not_use
0,DanielleTable-2021-09-03-16-10,5,0.12422,0.36667,0.18672,0.50278,3,1.0
1,DanielleTable-2021-09-03-16-10,5,0.31250,0.28889,0.36094,0.41528,3,1.0
2,DanielleTable-2021-09-03-16-10,5,0.52812,0.91111,0.59766,0.99722,3,1.0
3,DanielleTable-2021-09-03-16-10,5,0.43828,0.16667,0.48906,0.37500,2,1.0
4,DanielleTable-2021-09-03-16-10,5,0.16328,0.28611,0.20781,0.41806,3,1.0
...,...,...,...,...,...,...,...,...
213,Jenine-gigi-2020-01-10-15-03,175,0.84062,0.42778,0.90859,0.53194,3,1.0
214,Teachersattimeofincident-2022-08-26-17-08,8,0.95703,0.59861,0.99922,0.75972,2,1.0
215,Teachersattimeofincident-2022-08-26-17-08,10,0.93750,0.57639,0.98750,0.75833,1,1.0
216,Teachersattimeofincident-2022-08-26-17-08,11,0.94453,0.60417,0.97891,0.72500,1,1.0


In [12]:
combined_val_labels["video_id"].unique()

array(['DanielleTable-2021-09-03-16-10', 'Easton-2022-08-24-16-58',
       'Jack-ActivityExclusion-2022-07-14-15-30',
       'Jenine-gigi-2020-01-10-15-03',
       'Teachersattimeofincident-2022-08-26-17-08', '-5KQ66BBWC4',
       '-FaXLcSFjUI', '-IELREHX_js', '-OyDO1g74vc', '-XpUuIgyUHE',
       '-ZFgsrolSxo', '053oq2xB3oU', '0f39OWEqJ24', '0wBYFahr3uI',
       '1ReZIMmD_8E', '_-Z6wFjXtGQ', '_145Aa_xkuE', '_Ca3gOdOHxU',
       '_a9SWtcaNj8', '_ithRWANKB0', '_mAfwH6i90E'], dtype=object)

In [15]:
combined_val_frames["original_vido_id"].unique().tolist()

['DanielleTable-2021-09-03-16-10',
 'Easton-2022-08-24-16-58',
 'Jack-ActivityExclusion-2022-07-14-15-30',
 'Jenine-gigi-2020-01-10-15-03',
 'Teachersattimeofincident-2022-08-26-17-08',
 '1j20qq1JyX4',
 '_7oWZq_s_Sk',
 '_dBTTYDRdRQ',
 '_eBah6c5kyA']

In [17]:
combined_val_labels

,video_id,timestamp,xmin,ymin,xmax,ymax,classes,not_use
0,DanielleTable-2021-09-03-16-10,5,0.12422,0.36667,0.18672,0.50278,3,1.0
1,DanielleTable-2021-09-03-16-10,5,0.31250,0.28889,0.36094,0.41528,3,1.0
2,DanielleTable-2021-09-03-16-10,5,0.52812,0.91111,0.59766,0.99722,3,1.0
3,DanielleTable-2021-09-03-16-10,5,0.43828,0.16667,0.48906,0.37500,2,1.0
4,DanielleTable-2021-09-03-16-10,5,0.16328,0.28611,0.20781,0.41806,3,1.0
...,...,...,...,...,...,...,...,...
21003,_mAfwH6i90E,1794,0.04700,0.06200,0.95800,0.97000,2,1.0
21004,_mAfwH6i90E,1795,0.05800,0.05200,0.95000,0.98000,2,1.0
21005,_mAfwH6i90E,1796,0.00000,0.10100,0.93700,0.98200,2,1.0
21006,_mAfwH6i90E,1797,0.00700,0.01200,0.92400,0.98500,2,1.0


In [16]:
combined_val_labels[combined_val_labels["video_id"].isin(combined_val_frames["original_vido_id"].unique().tolist())]

,video_id,timestamp,xmin,ymin,xmax,ymax,classes,not_use
0,DanielleTable-2021-09-03-16-10,5,0.12422,0.36667,0.18672,0.50278,3,1.0
1,DanielleTable-2021-09-03-16-10,5,0.31250,0.28889,0.36094,0.41528,3,1.0
2,DanielleTable-2021-09-03-16-10,5,0.52812,0.91111,0.59766,0.99722,3,1.0
3,DanielleTable-2021-09-03-16-10,5,0.43828,0.16667,0.48906,0.37500,2,1.0
4,DanielleTable-2021-09-03-16-10,5,0.16328,0.28611,0.20781,0.41806,3,1.0
...,...,...,...,...,...,...,...,...
213,Jenine-gigi-2020-01-10-15-03,175,0.84062,0.42778,0.90859,0.53194,3,1.0
214,Teachersattimeofincident-2022-08-26-17-08,8,0.95703,0.59861,0.99922,0.75972,2,1.0
215,Teachersattimeofincident-2022-08-26-17-08,10,0.93750,0.57639,0.98750,0.75833,1,1.0
216,Teachersattimeofincident-2022-08-26-17-08,11,0.94453,0.60417,0.97891,0.72500,1,1.0


In [13]:
custom_val_frames["original_vido_id"].unique()

array(['DanielleTable-2021-09-03-16-10', 'Easton-2022-08-24-16-58',
       'Jack-ActivityExclusion-2022-07-14-15-30',
       'Jenine-gigi-2020-01-10-15-03',
       'Teachersattimeofincident-2022-08-26-17-08'], dtype=object)

In [14]:
ava_val_frames["original_vido_id"].unique()

array(['1j20qq1JyX4', '_7oWZq_s_Sk', '_dBTTYDRdRQ', '_eBah6c5kyA'],
      dtype=object)